## 概要

量子コンピュータは現在稼働中の古典コンピュータとは異なる基本単位の量子ビット
によって構成されており、量子ビットの性質を持ちいることで今までの
古典コンピュータでは不可能であった計算や情報処理が可能になることが期待されている。\
ただ量子コンピュータには実行上種々の制約が存在し、量子回路上で用いることのできる
行列がユニタリ行列に限られるのも制約の一つである。
この制約により非ユニタリ行列の期待値計算や
量子コンピュータ上での量子状態への作用は困難であり、解決すべき課題である。\
この問題に対する解決策の一つが本モジュールで扱う
"Block-Encoding"（以後BE）法[1]である。
BE法は非ユニタリ行列がPauli行列積の線型結合で表現できる場合、
構成するPauli行列積を制御ビットを介して作用させることで量子状態に
作用させることができる。
これにより量子回路上でも非ユニタリ行列を量子状態に作用させることができる。

このページではBE法についての理論的背景及び抱えている問題点について解説する。\
本モジュール`PItBE`を用いてBE法を実行する際の手順については[]()を参照すること。

## 理論的背景

BE法の理論的背景を説明する。前提として扱いたい$2^n$次正方行列の非ユニタリ行列を$\hat{A}$とし、$\hat{A}$は係数$\alpha_i$とユニタリ行列積$\hat{P}_i$を用いて
$$
\hat{A} = \sum_i^m\alpha_i\hat{P}_i
$$
と表現できるとする。\
（$m$は自然数、なお今回は後述する説明のために$m=2^k, k=1,2,3,\dots$とする。また$\alpha_i$は複素数、こちらも後述する説明のために規格化されている（$\sum_i^m|\alpha_i|^2=1$）とする。）\
そして非ユニタリ行列を作用させたい量子状態を
$$|\psi\rangle$$
とする。


まずBE法を実行する量子回路を紹介する。

![BE Circuit](./be_circ.png)

BE法における量子状態は作用させたい量子状態を格納したメイン量子ビットに加えて、メイン量子ビットに作用させる量子ゲート$\hat{P}_i$を制御するための補助量子ビットで構成されている。クロネッカー積で量子状態を表現すると次の式のようになる。
$$
|0\dots{}0\rangle_{\text{anci}}\otimes|\psi\rangle_{\text{main}}
$$
補助量子ビットの個数は$\log_2(m)$であり、今回の場合は$k$個である。\
また量子回路中に現れる量子ゲートは非ユニタリ行列を構成するユニタリ行列を量子状態に作用させる制御量子ゲート$\hat{P}_i$と、補助ビット部分のみに作用する量子ゲート$\hat{B}$及び$\hat{C}$の三種類に大別される。\
なお$\hat{B}$及び$\hat{C}$は$2^k$次正方ユニタリ行列であり、それぞれの要素は複素数であり、
$$
\begin{aligned}
    \hat{B}: \beta_{s,t}\\
    \hat{C}: \gamma_{s,t}
\end{aligned}
$$
とする。

ここからは量子回路の実行に伴って変化する量子状態を追いながら理論的背景を説明する。\
最初は補助量子ビット部分に量子ゲート$\hat{B}$を作用させる。この操作により補助量子ビットのとる量子状態は次のような重ね合わせ状態となる。
$$
|0\dots{}0\rangle_{\text{anci}}\otimes|\psi\rangle_{\text{main}}
\to
\left(\sum_i^m{\beta_{i,0}}|i\rangle_{\text{anci}}\right)\otimes|\psi\rangle_{\text{main}}
$$
ここで$|i\rangle$は$k$桁の二進数表記における整数$i$を表している。\
（例：$k=5$における$i=8$の場合$|i\rangle = |00100\rangle$）

次に補助量子ビットの値を制御ビットとする制御量子ゲート$\hat{P}_i$をメイン量子ビットに作用させていく。作用させる制御量子ゲートと補助量子ビットの値の対応関係は、補助量子ビットの値が$|i\rangle$の場合、$i$番目の量子ゲートを作用させるというものになっている。
$$
\left(\sum_i^m\beta_{i,0}|i\rangle_{\text{anci}}\right)\otimes|\psi\rangle_{\text{main}}
\to
\sum_i^m\beta_{i,0}|i\rangle_{\text{anci}}\otimes\hat{P}_i|\psi\rangle_{\text{main}}
$$
この操作により異なる量子ゲートを$|\psi\rangle$に作用させた状態が補助量子ビットの値によって区別された重ね合わせ状態として保存することができる。

制御量子ゲートの作用が完了後、補助量子ビット部分に作用させた量子ゲート$\hat{C}$を作用させる。作用後の量子状態は次のように変化する。
$$
\sum_i^m\beta_{i,0}|i\rangle_{\text{anci}}\otimes\hat{P}_i|\psi\rangle_{\text{main}}
\to
\sum_{i,j}^m\gamma_{j,i}\beta_{i,0}|j\rangle_{\text{anci}}\otimes\hat{P}_i|\psi\rangle_{\text{main}}
$$
このときダミーインデックス$j$のいずれかに対して次式
$$
\gamma_{j,i}\beta_{i,0} = t\alpha_i\quad(t>0)
$$
が成立するように$\gamma_{j,i}$及び$\beta_{i,0}$の値を決定すると、補助量子ビットが$|j\rangle$となる量子状態を射影測定することで$\hat{A}|\psi\rangle$についての情報を得ることができる。
$$
\begin{aligned}
    &\quad\sum_{i}^m\gamma_{j,i}\beta_{i,0}|j\rangle_{\text{anci}}\otimes\hat{P}_i|\psi\rangle_{\text{main}}\\
    &=\sum_{i}^mt\alpha_i|j\rangle_{\text{anci}}\otimes\hat{P}_i|\psi\rangle_{\text{main}}\\
    &=|j\rangle_{\text{anci}}\otimes\left(\sum_{i}^mt\alpha_i\hat{P}_i|\psi\rangle_{\text{main}}\right)\\
    &=|j\rangle_{\text{anci}}\otimes{}t\hat{A}|\psi\rangle_{\text{main}}\\
\end{aligned}
$$
なお実際には$j=0$とすることを推奨する。これは量子ゲート$\hat{B}$及び$\hat{C}$については行列成分が前述した条件を満たしていれば良いが、
$$
\beta_{i,0} = \sqrt{t\alpha_i}
$$
とすることにより
$$
\gamma_{0,i} = \sqrt{t\alpha_i}
$$
となり、
$$
\hat{C} = \hat{B}^t
$$
とすることができるためである。

射影測定時にBE法の実行結果を測定する確率$p_j$は次のように計算される。
$$
\begin{aligned}
    p_j 
    &= ||j\rangle_{\text{anci}}\otimes{}t\hat{A}|\psi\rangle_{\text{main}}|^2\\
    &= \langle{}j|j\rangle_{\text{anci}}\times{}t^2\langle{}\psi|\hat{A}^{\dag}\hat{A}|\psi\rangle_{\text{main}}\\
    &= t^2\langle{}\psi|\hat{A}^{\dag}\hat{A}|\psi\rangle_{\text{main}}\\
\end{aligned}
$$
ここで$\hat{A}$の固有ベクトル$|\phi_i\rangle$及び定数$\epsilon_i$により量子状態$|\psi\rangle$が
$$
|\psi\rangle=\sum_l\epsilon_l|\phi_l\rangle
$$
と書けるとすると$\hat{A}$の固有値$\lambda_i$を用いて
$$
\begin{aligned}
    p_j 
    &= t^2\langle{}\psi|\hat{A}^{\dag}\hat{A}|\psi\rangle_{\text{main}}\\
    &= t^2\sum_i|\epsilon_i\lambda_i|\phi_i\rangle|^2\\
    &= t^2\sum_i|\epsilon_i\lambda_i|^2
\end{aligned}
$$
となる。

## 問題点

BE法を実行する上では量子ゲート$\hat{B}$及び$\hat{C}$を実装する必要があるが、現在のところPauli回転ゲートを組み合わせることで量子ビットの値が全て0の値となる状態のみをとる量子状態から、任意の量子状態へと変換する量子ゲートを実装する方法は確立されていない。\
筆者は機械学習を応用することで実装できるのではないかと考えているが定かではない。

加えてBE法の実行には$\hat{A}$を構成するPauli行列積を作用させるための制限量子ゲート$\hat{P}_i$の実装も必要だが、複数の制御ビットが必要となる量子ゲートの実装はCNOTゲートを複数個必要となる。\
これと量子ゲート$\hat{B}$と$\hat{C}$を実装するのに必要な量子ゲート数を合わせて考えると、BE法の実行にはいわゆるNISQ型の量子コンピュータでは不十分である。\
BE法の本格的な実行には量子誤り訂正の実現した量子コンピュータか、量子ゲート数を削減させる方法の確立が必要であると筆者は考えている。

## 参考文献

[1] Andrew M Childs and Nathan Wiebe. “Hamiltonian simulation using linear combinations of unitary operations”. In: 
[arXiv preprint arXiv:1202.5822](https://arxiv.org/abs/1202.5822) (2012). 


BE法を提案した論文